<a href="https://colab.research.google.com/github/SojeongShin/NLP_2023/blob/Notion/4_2_BOW_%EA%B8%B0%EB%B0%98%EC%9D%98_%EB%AC%B8%EC%84%9C%EB%B6%84%EB%A5%98_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [자연어처리]
# 4주차(4-2). BOW 기반의 문서 분류(3)


# 202001581 신소정 <- 본인의 학번, 이름으로 변경하여 제출
* **모든 셀 실행** 후 제출하시기 바랍니다.
* **실습 (4-2-1, 4-2-2)**이 있습니다.

# 이제 Colab 사용에 모든 수강생분들이 어느정도 익숙해 졌을 거라 생각됩니다!
* (벌써 4주차 입니다. 시간 빠르죠?)
* **키보드**(+마우스)를 조금 더 사용해보도록 하겠습니다.
> ##### 1. **주석을 해제**하거나
> ##### 2. 다시 **따라 쳐보면서** 실습을 진행해 봅시다. (V)
> ##### 중간 중간 **'# ... 이 부분을 완성하시오'** 라는 부분의 코드를 완성해야 합니다.





## 3-2 & 4-1. 수업 내용 리뷰 시작
## 20 뉴스그룹 데이터 준비 및 특성 추출

 http://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

In [1]:
from sklearn.datasets import fetch_20newsgroups

# 20개의 토픽 중 선택하고자 하는 토픽을 리스트로 생성
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# 학습 데이터셋을 가져옴
# 메일 내용에서 hint가 되는 부분을 삭제 - 순수하게 내용만으로 분류
newsgroups_train = fetch_20newsgroups(subset='train', # ... 이 부분을 완성하시오
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

# 테스트 데이터셋을 가져옴
newsgroups_test = fetch_20newsgroups(subset='test', # ... 이 부분을 완성하시오
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Test set size:', len(newsgroups_test.data))
print('#Selected categories:', newsgroups_train.target_names)
print('#Train labels:', set(newsgroups_train.target))

#Train set size: 2034
#Test set size: 1353
#Selected categories: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#Train labels: {0, 1, 2, 3}


In [2]:
# 학습 데이터셋 문서
X_train = newsgroups_train.data
# 학습 데이터셋 레이블
y_train = newsgroups_train.target

# 테스트 데이터셋 문서
X_test = newsgroups_test.data
# 테스트 데이터셋 레이블
y_test = newsgroups_test.target

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 방식 / feature의 개수는 2000개를 사용
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5) # ... 이 부분을 완성하시오

# train set을 변환
X_train_tfidf = tfidf.fit_transform(X_train)  # ... 이 부분을 완성하시오
# test set을 변환
X_test_tfidf = tfidf.transform(X_test)  # ... 이 부분을 완성하시오

In [4]:
import numpy as np

def top10_features(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names_out())
    for i, category in enumerate(categories):
        # 역순으로 정렬하기 위해 계수에 음수를 취해서 정렬 후 앞에서부터 10개의 값을 반환
        top10 = np.argsort(-classifier.coef_[i])[:10]
        # 카테고리와 영향이 큰 특성 10개를 출력
        print("%s: %s" % (category, ", ".join(feature_names[top10])))

## 3-2 & 4-1. 수업 내용 리뷰 끝
---

## 4-2. 수업 내용 시작

## 4. 로지스틱 회귀분석을 이용한 문서 분류

### 릿지 회귀(ridge regression)


In [5]:
# 사이킷런의 RidgeClassifier 이용
from sklearn.linear_model import RidgeClassifier

# 릿지 분류기 선언
ridge_clf =RidgeClassifier() # ... 이 부분을 완성하시오
# 학습
ridge_clf.fit(X_train_tfidf, y_train) # ... 이 부분을 완성하시오

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.960
Test set score: 0.735


In [6]:
# def top10_features(classifier, vectorizer, categories): 를 이용하여 top10 feature를 확인하시오
top10_features(ridge_clf, tfidf, newsgroups_train.target_names) # ... 이 부분을 완성하시오

alt.atheism: bobby, religion, atheists, atheism, motto, punishment, satan, deletion, islamic, liar
comp.graphics: graphics, computer, 3d, file, 42, hi, image, using, screen, looking
sci.space: space, orbit, spacecraft, sci, moon, funding, nasa, 23, engineering, nick
talk.religion.misc: christian, blood, christians, fbi, order, objective, hudson, children, abortion, dead


In [10]:
# alpha 값을 0.1로 할 경우 -> 과적합 심화, 성능 다운
ridge_clf = RidgeClassifier(alpha=0.1) # 릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.977
Test set score: 0.673


In [11]:
top10_features(ridge_clf, tfidf, newsgroups_train.target_names)

alt.atheism: policy, risk, species, iii, liar, middle, sea, motto, contradictory, everything
comp.graphics: 42, split, site, 01, graphics, six, requires, edges, computer, main
sci.space: 500, tool, george, longer, wall, star, 23, expected, funding, allen
talk.religion.misc: blood, muhammad, moment, dead, decenso, deity, creation, clearly, 26, brought


In [12]:
from sklearn.model_selection import train_test_split

# 8:2의 비율로 나누기
X_train_ridge, X_val_ridge, y_train_ridge, y_val_ridge = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42) # ... 이 부분을 완성하시오

In [13]:
# ... 이 부분을 완성하시오 (전체 코드 작성)
# 설명: train에 대해 학습하고, validation에 대한 score를 재는 코드를 작성하시오.

# 1. alpha=0.1인 분류기 선언
ridge_clf = RidgeClassifier(alpha=0.1)
# 2. X_train_ridge에 대해 학습
ridge_clf.fit(X_train_ridge, y_train_ridge)
# 3. X_val_ridge에 대해 score 확인
ridge_clf.score(X_val_ridge, y_val_ridge)

0.7248157248157249

In [16]:
# ... 이 부분을 완성하시오 (전체 코드 작성)
# 설명: alpha를 0.1부터 10까지 0.1씩 증가시켜 학습시키고, score를 확인하는 코드를 작성하시오. range는 int단위
 # (hint: np.arange(start, stop, step))
# 위의 코드에서 alpha=0.1을 변경시켜가면서 score 값을 확인
import numpy as np
# 1. for 문 작성
for alpha in np.arange(0.1, 10, 0.1):
    # 2. 위의 코드 3줄 복사

    # 1. alpha 증가하는 분류기 선언
    ridge_clf = RidgeClassifier(alpha)
    # 2. X_train_ridge에 대해 학습
    ridge_clf.fit(X_train_ridge, y_train_ridge)
    # 3. X_val_ridge에 대해 score 확인
    score = ridge_clf.score(X_val_ridge, y_val_ridge)
    print(score)

0.7248157248157249
0.7714987714987716
0.7862407862407862
0.800982800982801
0.8034398034398035
0.800982800982801
0.8034398034398035
0.8083538083538083
0.8083538083538083
0.8132678132678133
0.8157248157248157
0.8206388206388207
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8255528255528255
0.8255528255528255
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8206388206388207
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8230958230958231
0.8157248157248157
0.8157248157248157
0.8157248157248157
0.8157248157248157
0.8108108108108109
0.8108108108108109
0.8108108108108109
0.8108108108108109
0.8108108108108109
0.8083538083538083
0.8083538083538083
0.8034398034398035
0.800982800982801
0.800982800982801
0.7985257985257985
0.7985257985257985
0.7985257985257985
0.7985257985257985
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.800982800982801
0.8034398034398035
0.8034398034398035
0.800

In [17]:
# ... 이 부분을 완성하시오 (전체 코드 작성)
# alpha를 0.1부터 10까지 0.1씩 증가시키면서, score 값을 최대로 만드는 alpha 값을 max_alpha에 저장하시오
# 위의 코드에서 score 값을 비교하여 저장하는 코드 추가

# 1. max_alpha 초기값 선언
max_alpha = 0
max_score = 0

# 2. for 문 작성
for alpha in np.arange(0.1, 10, 0.1):
    # 1. alpha 증가 분류기 선언
    ridge_clf = RidgeClassifier(alpha=alpha)
    # 2. X_train_ridge에 대해 학습
    ridge_clf.fit(X_train_ridge, y_train_ridge)
    # 3. X_val_ridge에 대해 score 확인
    score = ridge_clf.score(X_val_ridge, y_val_ridge)

    # 4. if 문을 통한 비교
    if score > max_score:
      max_score = score
      max_alpha = alpha
        # 5. max_alpha 저장

print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

Max alpha 1.600 at max validation score 0.826


In [19]:
# 릿지 분류기 선언
# 위의 셀에서 얻은 최대 alpha 값을 이용하시오
ridge_clf = RidgeClassifier(alpha=1.6)
# ridge_clf = RidgeClassifier(alpha=???) # ... 이 부분을 완성하시오
ridge_clf.fit(X_train_tfidf, y_train) # 학습

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.947
Test set score: 0.739


0.977 -> 0.947


0.673 -> 0.739

In [20]:
top10_features(ridge_clf, tfidf, newsgroups_train.target_names)

alt.atheism: bobby, religion, atheism, atheists, motto, punishment, islam, deletion, islamic, satan
comp.graphics: graphics, computer, 3d, file, image, hi, 42, using, screen, looking
sci.space: space, orbit, nasa, spacecraft, moon, sci, launch, flight, funding, idea
talk.religion.misc: christian, christians, fbi, blood, order, jesus, objective, children, christ, hudson


## 5. 결정트리 등을 이용한 기타 문서 분류 방법

1. https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
2. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
3. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [21]:
# DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=7)
tree.fit(X_train_tfidf, y_train)

print('#Decision Tree train set score: {:.3f}'.format(tree.score(X_train_tfidf, y_train)))
print('#Decision Tree test set score: {:.3f}'.format(tree.score(X_test_tfidf, y_test)))

#Decision Tree train set score: 0.977
#Decision Tree test set score: 0.536


In [22]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(random_state=7)
forest.fit(X_train_tfidf, y_train)

print('#Random Forest train set score: {:.3f}'.format(forest.score(X_train_tfidf, y_train)))
print('#Random Forest test set score: {:.3f}'.format(forest.score(X_test_tfidf, y_test)))

#Random Forest train set score: 0.977
#Random Forest test set score: 0.685


In [23]:
# GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=7)
gb.fit(X_train_tfidf, y_train)

print('#Gradient Boosting train set score: {:.3f}'.format(gb.score(X_train_tfidf, y_train)))
print('#Gradient Boosting test set score: {:.3f}'.format(gb.score(X_test_tfidf, y_test)))

#Gradient Boosting train set score: 0.933
#Gradient Boosting test set score: 0.696


In [24]:
gb.feature_importances_

array([0.        , 0.        , 0.00018793, ..., 0.00321606, 0.        ,
       0.        ])

In [25]:
sorted(gb.feature_importances_, reverse=True)[:40]

[0.12606815556503398,
 0.07955738535484687,
 0.02354325879146636,
 0.023046507462732756,
 0.0205558124023883,
 0.020063273938036065,
 0.01814149212443511,
 0.01770949154089045,
 0.016857094927865725,
 0.015308260768876465,
 0.015023452519852164,
 0.013797145686816017,
 0.010426583770790022,
 0.010376173054679735,
 0.010225361352506715,
 0.010061136447910934,
 0.009663039306452306,
 0.009660082651339493,
 0.00940203835982551,
 0.008981046753380314,
 0.008827368636033589,
 0.007819020701620829,
 0.007751440501423645,
 0.007515358150120413,
 0.007255118998822582,
 0.007182617149762632,
 0.007074189152295397,
 0.0068990602775437955,
 0.006698870635101267,
 0.006529364772733122,
 0.006245749237205102,
 0.005513872126424359,
 0.004918770389689076,
 0.004805134811116731,
 0.004767940213268277,
 0.004624503816321026,
 0.004608372640219624,
 0.004571604182502272,
 0.004415620817426219,
 0.004334578233902469]

In [26]:
sorted_feature_importances = sorted(zip(tfidf.get_feature_names_out(), gb.feature_importances_), key=lambda x: x[1], reverse=True)
for feature, value in sorted_feature_importances[:40]:
    print('%s: %.3f' % (feature, value), end=', ')

space: 0.126, graphics: 0.080, atheism: 0.024, thanks: 0.023, file: 0.021, orbit: 0.020, jesus: 0.018, god: 0.018, hi: 0.017, nasa: 0.015, image: 0.015, files: 0.014, christ: 0.010, moon: 0.010, bobby: 0.010, launch: 0.010, christian: 0.010, looking: 0.010, atheists: 0.009, christians: 0.009, fbi: 0.009, 3d: 0.008, you: 0.008, not: 0.008, islamic: 0.007, religion: 0.007, spacecraft: 0.007, flight: 0.007, computer: 0.007, islam: 0.007, ftp: 0.006, color: 0.006, software: 0.005, atheist: 0.005, card: 0.005, people: 0.005, koresh: 0.005, his: 0.005, kent: 0.004, sphere: 0.004, 

# [[실습 4-2-1]]
# coef_의 결과와 feature_importances_ 결과 비교를 통해, 회귀분석 방법과 트리 기반의 분류 방식의 차이점 알아보기
* coef_의 결과(여러 분류기 중 1개 이상)와 feature_importances_ 결과(여러 분류기 중 1개 이상) 복사/붙여넣기
* 비교를 통해 어떤 차이점이 있는 것 같은지 분석해보시오.

- coef_ Result

alt.atheism: bobby, religion, atheists, atheism, motto, punishment, satan, deletion, islamic, liar
comp.graphics: graphics, computer, 3d, file, 42, hi, image, using, screen, looking
sci.space: space, orbit, spacecraft, sci, moon, funding, nasa, 23, engineering, nick
talk.religion.misc: christian, blood, christians, fbi, order, objective, hudson, children, abortion, dead

- feature_importances_ Result

space: 0.126, graphics: 0.080, atheism: 0.024, thanks: 0.023, file: 0.021, orbit: 0.020, jesus: 0.018, god: 0.018, hi: 0.017, nasa: 0.015, image: 0.015, files: 0.014, christ: 0.010, moon: 0.010, bobby: 0.010, launch: 0.010, christian: 0.010, looking: 0.010, atheists: 0.009, christians: 0.009, fbi: 0.009, 3d: 0.008, you: 0.008, not: 0.008, islamic: 0.007, religion: 0.007, spacecraft: 0.007, flight: 0.007, computer: 0.007, islam: 0.007, ftp: 0.006, color: 0.006, software: 0.005, atheist: 0.005, card: 0.005, people: 0.005, koresh: 0.005, his: 0.005, kent: 0.004, sphere: 0.004,

###위와 같이 coef_ 결과는 각 카테고리별 빈도수가 높은 feature들 순으로 나열되어 있어 카테고리 별 영향력이 큰 단어 순으로 나누어 보기 편하다.

###feature_importances의 결과는 카테고리 전체의 feature들 각각이 모집단이 되므로 카테고리별 빈도수가 높은 단어들을 나누어 보기가 어렵다.

---

## 8. 한글 문서의 분류
### 입력 값으로 리뷰가 들어오면, 어떤 영화에 대한 리뷰인지 예측하는 알고리즘을 만들어 보자!
* '신과함께, 택시운전사, 인피니티 워, 범죄도시, 곤지암, 라라랜드, 코코' 중 하나로 분류
* **입력**: 몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.
* **출력**: 인피니티 워

In [27]:
import pandas as pd
df = pd.read_csv('./daum_movie_review.csv')
df.head(5) # 처음부터 다섯개까디지 출력

,review,rating,date,title
0,돈 들인건 티가 나지만 보는 내내 하품만,1,2018.10.29,인피니티 워
1,몰입할수밖에 없다. 어렵게 생각할 필요없다. 내가 전투에 참여한듯 손에 땀이남.,10,2018.10.26,인피니티 워
2,이전 작품에 비해 더 화려하고 스케일도 커졌지만.... 전국 맛집의 음식들을 한데 ...,8,2018.10.24,인피니티 워
3,이 정도면 볼만하다고 할 수 있음!,8,2018.10.22,인피니티 워
4,재미있다,10,2018.10.20,인피니티 워


In [28]:
df.tail(5)

,review,rating,date,title
14720,어른들을 위한 동화 정말 오랜만에 좋은 애니를 보았습니다 가족의 소중...,10,2018.01.12,코코
14721,디즈니는 못해도 본전은 한다.,7,2018.01.12,코코
14722,가족을 위한 영화... 괜찮은 영화.~~~,8,2018.01.12,코코
14723,간만에 제대로 잘짜여진 각본의 영화를 봤네 여운이 아직도 남아~어른을 위한 애니~,10,2018.01.12,코코
14724,한국개봉을 눈빠지게 기다린 보람이있다 깨우치는게 많은 영화,10,2018.01.12,코코


In [29]:
# review -> x, title-> y
df.title.value_counts()

신과함께      4947
택시운전사     2322
인피니티 워    2042
범죄도시      1939
곤지암       1547
라라랜드      1150
코코         778
Name: title, dtype: int64

In [30]:
df.shape

(14725, 4)

In [31]:
from sklearn.model_selection import train_test_split

# split data and labels into a training and a test set
# 비율을 지정하지 않으면 75:25로 분할됨
# train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)를 참고
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0) # ... 이 부분을 완성하시오

print('#Train set size:', len(X_train)) # 실제로 몇개의 특성이 사용되었는지 확인
print('#Test set size:', len(X_test))

#Train set size: 11043
#Test set size: 3682


In [32]:
! pip install konlpy  # 한글 처리

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 48.0 MB/s eta 0:00:00


In [33]:
from konlpy.tag import Okt # konlpy에서 Twitter 형태소 분석기를 import

okt = Okt()

print(okt.morphs(X_train[1])) # 둘째 리뷰에 대해 형태소 단위로 tokenize
print(okt.nouns(X_train[1])) # 둘째 리뷰에서 명사만 추출

['몰입', '할수밖에', '없다', '.', '어렵게', '생각', '할', '필요없다', '.', '내', '가', '전투', '에', '참여', '한', '듯', '손', '에', '땀', '이남', '.']
['몰입', '생각', '내', '전투', '참여', '듯', '손', '땀', '이남']


In [34]:
# 약간의 시간이 소요됩니다! (약 1분 30초)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Okt 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)  # mytokenizer 사용 -> 시간 오래걸림

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [35]:
# logistic regression 분류기 선언
LR_clf = LogisticRegression(max_iter=1000)
# 분류기 학습
LR_clf.fit(X_train_tfidf, y_train)

print('#Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data 예측정확도
print('#Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data 예측정확도

#Train set score: 0.756
#Test set score: 0.694


In [38]:
top10_features(LR_clf, tfidf, sorted(df.title.unique()))

곤지암: 공포영화, 공포, 귀신, 곤지암, 무서움, 소리, 기담, 개, 공포물, 장난
라라랜드: 음악, 뮤지컬, 꿈, 사랑, 노래, 엔딩, 여운, 인생, 재즈, 환상
범죄도시: 마동석, 윤계상, 조선족, 액션, 조연, 조폭, 범죄, 마블리, 형사, 폭력
신과함께: 원작, 차태현, 웹툰, 신파, 김동욱, 판타지, 신, 천만, 지옥, 하정우
인피니티 워: 마블, 노스, 어벤져스, 히어로, 번역, 토르, 편, 결말, 다음, 영웅
코코: 디즈니, 애니, 겨울왕국, 감동, 코코, 픽사, 애니메이션, 가족, 어른, 노래
택시운전사: 광주, 송강호, 역사, 전두환, 택시, 가슴, 진실, 민주주의, 감동, 추격


In [39]:
# 첫 번째 리뷰에 대한 예측 값 살펴보기
print('첫 번째 리뷰 텍스트: ', X_test.iloc[0])
print('특성으로 추출된 토큰: ', okt.nouns(X_test.iloc[0]))
predict = LR_clf.predict(X_test_tfidf[0])
print('예측된 영화 제목: ', predict[0])
print('실제 영화 제목: ', y_test.iloc[0])

첫 번째 리뷰 텍스트:  오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.
특성으로 추출된 토큰:  ['만', '영화', '다음', '더', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  범죄도시


In [40]:
for i in range(10):
    print(i, '번째 리뷰 텍스트: ', X_test.iloc[i])
    print('특성으로 추출된 토큰: ', okt.nouns(X_test.iloc[i]))
    predict = LR_clf.predict(X_test_tfidf[i])
    print('예측된 영화 제목: ', predict[0])
    print('실제 영화 제목: ', y_test.iloc[i])
    print()

0 번째 리뷰 텍스트:  오랜만에 잼나는 영화 봤습니다.  다음에 더 재미있는 영화 기대하겠습니다.
특성으로 추출된 토큰:  ['만', '영화', '다음', '더', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  범죄도시

1 번째 리뷰 텍스트:  조연들이 눈에 박힌다. 간만에 집중 ㅎ
특성으로 추출된 토큰:  ['조연', '눈', '간만', '집중']
예측된 영화 제목:  범죄도시
실제 영화 제목:  범죄도시

2 번째 리뷰 텍스트:  대감동을 선사. 인사이드 아웃을 잇는 픽사의 감동스토리. 신과함께의 멕시코판이라고나할까요??
특성으로 추출된 토큰:  ['감동', '선사', '인사', '이드', '아웃', '픽사', '감동', '스토리', '신', '의', '멕시코', '판이', '나', '요']
예측된 영화 제목:  코코
실제 영화 제목:  코코

3 번째 리뷰 텍스트:  돈이 안아까웠던 영화ᆞᆞ  정말 좋았다
특성으로 추출된 토큰:  ['돈', '영화', '정말']
예측된 영화 제목:  신과함께
실제 영화 제목:  신과함께

4 번째 리뷰 텍스트:  역시 김용화감독이 영화는 잘 만들어요. 이제 VFX 제작 부문도 헐리우드 수준 이상입니다.
특성으로 추출된 토큰:  ['역시', '김용화', '감독', '영화', '이제', '제작', '부문', '헐리우드', '수준', '이상']
예측된 영화 제목:  신과함께
실제 영화 제목:  신과함께

5 번째 리뷰 텍스트:  민주화를 위해 힘써주신 분들께 감사하는 마음으로 살아야겠다.
특성으로 추출된 토큰:  ['민주화', '위해', '분', '마음']
예측된 영화 제목:  택시운전사
실제 영화 제목:  택시운전사

6 번째 리뷰 텍스트:  잠만 자다 왔음
특성으로 추출된 토큰:  ['잠']
예측된 영화 제목:  신과함께
실제 영화 제목:  신과함께

7 번째 리뷰 텍스트:  오랜만에 잼있고 좋은 영화를 봤다
특성으로 추출된 토큰:  ['만', '영화']
예측된 영화 제목:  신과함께
실제 영화 제목:  

# [[실습 4-2-2]]
* RidgeClassifier (alpha=0.1)를 이용하여 분류할 경우, Train set score와 Test set score는 얼마인가?
* (가능한 경우에만 시도) 이 데이터 셋에서는 valdation에 대한 최적의 alpha는 몇인가? 0.1과 10 사이에서 0.1씩 증가시키며 찾아보시오.


In [64]:
# RidgeClassifier (alpha=0.1)를 이용하여 분류
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


# 릿지 분류기 선언
ridge_clf =RidgeClassifier(alpha=0.1)
# 영화리뷰 파일 읽어오기
df = pd.read_csv('./daum_movie_review.csv')

# split data and labels into a training and a test set
# 비율을 지정하지 않으면 75:25로 분할됨
# train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)를 참고
X_train, X_test, y_train, y_test = train_test_split(df.review, df.title, random_state=0)

# Okt 형태소분석기에서 명사만 추출하는 함수를 tokenizer로 이용
tfidf = TfidfVectorizer(tokenizer=okt.nouns, max_features=2000, min_df=5, max_df=0.5)

X_train_tfidf = tfidf.fit_transform(X_train) # train data 변환 -> tfidf vector
X_test_tfidf = tfidf.transform(X_test) # test data 변환 -> tfidf vector


# 학습
ridge_clf.fit(X_train_tfidf, y_train)

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Train set score: 0.788
Test set score: 0.679


In [65]:
# alpha를 0.1부터 10까지 0.1씩 증가시키면서, score 값을 최대로 만드는 alpha 값을 max_alpha에 저장
# 위의 코드에서 score 값을 비교하여 저장하는 코드 추가

# 1. max_alpha 초기값 선언
max_alpha = 0
max_score = 0

# 2. for 문 작성
for alpha in np.arange(0.1, 10, 0.1):
    # 1. alpha 증가 분류기 선언
    ridge_clf = RidgeClassifier(alpha=alpha)
    # 2. X_train_ridge에 대해 학습
    ridge_clf.fit(X_train_tfidf, y_train)
    # 3. X_val_ridge에 대해 score 확인
    score = ridge_clf.score(X_test_tfidf, y_test)

    # 4. if 문을 통한 비교
    if score > max_score:
      max_score = score
      max_alpha = alpha
        # 5. max_alpha 저장

print('Max alpha {:.3f} at max validation score {:.3f}'.format(max_alpha, max_score))

Max alpha 3.400 at max validation score 0.699


In [66]:
ridge_clf =RidgeClassifier(alpha=3.4)

# 학습
ridge_clf.fit(X_train_tfidf, y_train)

print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))

Train set score: 0.757
Test set score: 0.699


(0.1과 10사이에서) 최적의 alpha 값과 해당 alpha 값을 이용하여 학습했을 때, Train set score와 Test set score는?

alpha = 0.1 ->
Train set score: 0.788
Test set score: 0.679

alpha = 3.4 ->
Train set score: 0.757
Test set score: 0.699

# [파일] -> [다운로드] -> [.ipynb 다운로드]
# 제출 후 **구글 계정 로그아웃** 잘 하시기 바랍니다.
# 4-2. 수업 내용 끝